<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Попробуем построить модель со значением метрики качества *F1* не менее 0.75. 

**План действий по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# помним про PEP-8
# импорты из стандартной библиотеки
import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.core.display import display, HTML

In [3]:
display(HTML("<style>.container { width:88% !important; }</style>"))

In [4]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import nltk
import lightgbm as lgb

#импортируем встроенный модуль для работы с регулярными выражениями (regular expressions)
import re
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
tqdm.pandas()

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split
)

from time import time

In [5]:
#Подгрузим необходимые файлы для английской библиотеки лемматизатора:

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Games\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Games\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
#константы

RANDOM_STATE = 12345
CV = 5 
VERBOSE = 1
N_JOBS = -1

In [7]:
try:
    forsen = pd.read_csv('/datasets/toxic_comments.csv')
except:
    forsen = pd.read_csv('C:/Users/Games/Downloads/toxic_comments.csv')

Посмотрим общую информацию по датасету.

In [8]:
forsen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [9]:
forsen.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Сразу бросается в глаза лишний столбец 'Unnamed: 0', который, по всей видимости, дублирует индексы. Избавимся от него т.к. он не несёт никакой смысловой нагрузки.

In [10]:
forsen = forsen.drop('Unnamed: 0', axis=1)

In [11]:
forsen.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [12]:
forsen.duplicated().sum()

0

Ни пропусков, ни дубликатов в датасете не имеется.

Для работы с регулярными выражениями в Python есть встроенный модуль re (сокр. от regular expressions). Создадим функцию, которая очистит текст для будущей лемматизации.  
А также создадим функцию для токенизации и лемматизации текста.

In [13]:
def clear_text(text):
    
    new_text = re.sub(r'[^a-zA-Z ]', ' ', text.lower())
    new_text = new_text.split()
    new_text = " ".join(new_text)
    return new_text

In [14]:
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

2023-08-12 16:49:12.151049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-08-12 16:49:12.151218: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-12 16:49:16.829273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-08-12 16:49:16.830353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2023-08-12 16:49:16.831266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2023-08-12 16:49:16.832446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB ? eta -:--:--
                                              0.1/12.8 MB 1.3 MB/s eta 0:00:10
     -                                        0.4/12.8 MB 3.3 MB/s eta 0:00:04
     --                                       0.7/12.8 MB 4.4 MB/s eta 0:00:03
     ----                                     1.4/12.8 MB 6.6 MB/s eta 0:00:02
     ------                                   2.0/12.8 MB 7.6 MB/s eta 0:00:02
     -------                                  2.6/12.8 MB 8.2 MB/s eta 0:00:02
     ---------                                3.1/12.8 MB 8.7 MB/s eta 0:00:02
     -----------                              3.7/12.8 MB 9.0 MB/s eta 0:00:02
     -------------                            4.2/12.8 MB 9.3 MB/s eta 0:00:01
     --------------                           4.8/12.8 MB 9.5 MB/s eta 0:00:01
     ----------------                         5.3/12.8 MB 9.8 MB/s

In [15]:
def lemmatize(text):
    
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [16]:
#lemmatizer = WordNetLemmatizer()
#
#def lemmatize(text):
#    
#    word_list = nltk.word_tokenize(text)
#    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

Сразу создадим доп. столбец в датасете с обработанными комментариями (правками).

In [17]:
forsen['lemm_text'] = forsen['text'].progress_apply(clear_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [18]:
forsen['lemm_text'] = forsen['lemm_text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [19]:
forsen['lemm_text']

0         explanation why the edit make under my usernam...
1         d aww he match this background colour I m seem...
2         hey man I m really not try to edit war it s ju...
3         more I can t make any real suggestion on impro...
4         you sir be my hero any chance you remember wha...
                                ...                        
159287    and for the second time of ask when your view ...
159288    you should be ashamed of yourself that be a ho...
159289    spitzer umm there s no actual article for pros...
159290    and it look like it be actually you who put on...
159291    and I really don t think you understand I come...
Name: lemm_text, Length: 159292, dtype: object

Посмотрим на соотношение объёма положительного класса к отрицательному.

In [20]:
forsen['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [21]:
print(f"Процент объектов класса 1 к общему объёму датасета: {(sum(forsen['toxic']) / len(forsen) * 100):.2f}%")

Процент объектов класса 1 к общему объёму датасета: 10.16%


Чтобы алгоритмы умели определять тематику и тональность текста, их нужно обучить на корпусе. Это набор текстов, в котором эмоции и ключевые слова уже размечены.

Разделим датасет на тестовую и тренировочную выборку, размер тестовой выборки - 10% от общих данных:

In [22]:
train_features, test_features, train_target, test_target = train_test_split(
    forsen.drop('toxic', axis=1),
    forsen['toxic'],
    test_size=0.1,
    random_state=RANDOM_STATE,
    stratify=forsen['toxic'] # стратифицируем текст, чтобы выборки были более сбалансированы
)

# вытаскиваем корпусы
corpus_train = train_features['lemm_text']
corpus_test = test_features['lemm_text']
corpus_train

92651     a thing at time the moans of birkin bardot be ...
57776     re suicide I m go to take my advice and shrug ...
155506                                       archive to dec
14143     hello the tag be add because the article start...
128218    good morning look where the satanic comment ip...
                                ...                        
9205      yet again I have repetedlty ask you not to add...
19763     my edit be an addition why would I modify the ...
136558    a reference that may interest you hello zleitz...
75564     I m not try to make a point or anything except...
55751     talk grasshopper scout I move your comment fro...
Name: lemm_text, Length: 143362, dtype: object

In [23]:
corpus_train.shape, corpus_test.shape

((143362,), (15930,))

Мешок слов учитывает частоту употребления слов. Оценка важности слова определяется величиной TF-IDF.  
То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.  
Воспользуемся TfidfVectorizer и, чтобы почистить мешок слов, добавим в него стоп-слова:

In [24]:
stopwords = set(nltk_stopwords.words('english'))

In [25]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) # подгружаем счетчик и задаём стоп-слова
tf_idf_train = count_tf_idf.fit_transform(corpus_train) # обучаем и трансформируем
tf_idf_test = count_tf_idf.transform(corpus_test) # трансформируем тестовую без обучения

print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (143362, 142726)
Размер матрицы: (15930, 142726)


## Обучение

Напишем функцию для кросс-валидации наших моделей.

In [26]:
warnings.filterwarnings("ignore")

In [27]:
def train_model(model, parameters):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('model', model)])
    model_random = RandomizedSearchCV(
        pipeline,
        cv=CV,
        n_jobs=N_JOBS,
        param_distributions=parameters,
        scoring='f1',
        verbose=VERBOSE
    )
    
    start = time()
    model_random.fit(train_features['lemm_text'], train_target)
    print('RandomizedSearchCV подбирал параметры %.2f секунд' %(time() - start))
    
    # высчитаем метрику
    f1 = model_random.best_score_
    
    print('Лучшие параметры:', model_random.best_params_)
    print('F1 обученной модели:', f1)

    return model_random, f1

**Logistic Regression**

In [28]:
param_grid = {
    "model__C": [0.01, 0.1, 1, 10, 100]
}

In [29]:
lr = LogisticRegression()

In [30]:
lr_random, lr_f1 = train_model(lr, param_grid)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
RandomizedSearchCV подбирал параметры 127.93 секунд
Лучшие параметры: {'model__C': 10}
F1 обученной модели: 0.7882215790349266


**Random Forest Classifier**

In [31]:
param_grid = {
    "model__max_depth": [300, 310],
    "model__n_estimators": [12, 14],
}

In [32]:
rfc = RandomForestClassifier()

In [33]:
rfc_random, rfc_f1 = train_model(rfc, param_grid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
RandomizedSearchCV подбирал параметры 1028.25 секунд
Лучшие параметры: {'model__n_estimators': 14, 'model__max_depth': 300}
F1 обученной модели: 0.5620716208369567


**LightGBM**

In [34]:
%%time

rand_lgbm_param = {
    "model__max_depth": [15, 25],
    "model__learning_rate": [0.1, 0.3]
}

gbm = lgb.LGBMClassifier()

gbm_random, gbm_f1 = train_model(gbm, rand_lgbm_param)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 14567, number of negative: 128795
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.082222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 615353
[LightGBM] [Info] Number of data points in the train set: 143362, number of used features: 10710
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101610 -> initscore=-2.179463
[LightGBM] [Info] Start training from score -2.179463
RandomizedSearchCV подбирал параметры 604.07 секунд
Лучшие параметры: {'model__max_depth': 25, 'model__learning_rate': 0.3}
F1 обученной модели: 0.7773540654242062
CPU times: t

Посмотрели три модели, ниже напишем выводы.

## Выводы

In [35]:
result = pd.DataFrame(
    [lr_f1, rfc_f1, gbm_f1], 
    index=['LogisticRegression', 'RandomForestClassifier', 'LGBM'], 
    columns=['F1']
)
result

,F1
LogisticRegression,0.788222
RandomForestClassifier,0.562072
LGBM,0.777354


Из трёх моделей лучшую метрику показала логистическая регрессия. Далее протестируем её с уже известными нам наилучшими гиперпараметрами на тестовой выборке.

In [37]:
%%time

lr_prediction = lr_random.predict(test_features['lemm_text'])
metric_test = f1_score(test_target, lr_prediction)
print('F1 лучшей модели на тестовой выборке:', metric_test)

F1 лучшей модели на тестовой выборке: 0.7983651226158036
CPU times: total: 844 ms
Wall time: 884 ms


Значение метрики на тестовой выборке (0,80) получилось выше требуемого порога в 0,75. Задание выполнено.

**Вывод**

Загрузив датасет, мы почистили тексты правок и лемматизировали их. Обработанные данные поместили в новый столбец датасета.  
Чтобы алгоритмы умели определять тематику и тональность текста, мы создали признаки и обучили их на корпусе. Это набор текстов, в котором эмоции и ключевые слова уже размечены. Перед этим мы разделили датасет на тренировочную и тестовую выборки.

Затем мы написали функцию для кросс-валидации наших моделей. Выбор пал на три модели:
* Логистическая регрессия
* Случайный лес
* LightGBM (градиентный бустинг)

Наилучшую метрику показала первая, 0,79.

Далее мы перешли непосредственно к тестированию лучшей модели на тестовой выборке, в результате чего **получили метрику F1 выше, чем того от нас требовалось, а именно 0,80 при необходимом минимальном пороге в 0,75.**